In [90]:
pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [91]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

In [96]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.evaluation import load_evaluator

embedding_function = OpenAIEmbeddings()
evaluator = load_evaluator("pairwise_embedding_distance")

words = ("apple", "banana")
x = evaluator.evaluate_string_pairs(prediction=words[0], prediction_b=words[1])
print(f"Comparing ({words[0]}, {words[1]}): {x}")

words = ("apple", "iphone")
x = evaluator.evaluate_string_pairs(prediction=words[0], prediction_b=words[1])
print(f"Comparing ({words[0]}, {words[1]}): {x}")

words = ("apple", "apple")
x = evaluator.evaluate_string_pairs(prediction=words[0], prediction_b=words[1])
print(f"Comparing ({words[0]}, {words[1]}): {x}")

Comparing (apple, banana): {'score': 0.09725941975023544}
Comparing (apple, iphone): {'score': 0.09710853291781563}
Comparing (apple, apple): {'score': -1.1102230246251565e-15}


In [92]:
!python project.py sapiens.pdf 

Split 1 documents into 111 chunks.
The

Present

Humans transcend the boundaries of planet Earth. Nuclear weapons

threaten the survival of humankind. Organisms are increasingly shaped

by intelligent design rather than natural selection.

The

Intelligent design becomes the basic principle of life? Homo sapiens is

Future

replaced by superhumans?

Part One

The Cognitive Revolution

1. A human handprint made about 30,000 years ago, on the wall of the Chauvet-Pont-d’Arc Cave in
{'source': 'data/books/sapiens.pdf', 'start_index': 4136}
Saved 111 chunks to chroma.


In [93]:
!python main.py  "Why humans run the world?"

0.7525280143990998
Human: 
Answer the question based only on the following context:

We assume that a large brain, the use of tools, superior learning abilities and complex social structures are huge advantages. It seems self-evident that these have made humankind the most powerful animal on earth. But humans enjoyed all of these advantages for a full 2 million years during which they remained weak and marginal creatures. Thus humans who lived a million years ago, despite their big brains and sharp stone tools, dwelt in constant fear of predators, rarely hunted large game, and

---

What was the Sapiens’ secret of success? How did we manage to settle so rapidly in so many distant and ecologically di(cid:643)erent habitats? How did we push all other human species into oblivion? Why couldn’t even the strong, brainy, cold- proof Neanderthals survive our onslaught? The debate continues to rage. The most likely answer is the very thing that makes the debate possible: Homo sapiens conquered 